In [0]:
import numpy as np
import pyspark as ps
import pyspark.pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql import SparkSession, Window
from pyspark.ml.linalg import Vectors

builder = SparkSession.builder
builder.config("spark.sql.ansi.enabled", "false")
builder.config("spark.sql.execution.arrow.pyspark.enabled", "true")
#builder.config('spark.executor.memory', '8g')
#builder.config('spark.driver.memory', '8g')
#builder.config("spark.driver.maxResultSize", "0")
spark = builder.getOrCreate()

#Chargement
df = pd.read_csv("/Volumes/alpha_cloud_ai_workspace/default/exp-data/cf_asset_mort_init.csv", sep=";")

#Ajout Interet
df['tx_interet'] = df['taux_hyp'] / df['frequence']

#Ajout des 1200 Cashflow
colones = dict()
for i in range(1200):
    colones[f'cash_flow_{i+1}'] = 0
df = df.reindex(columns=list(df.columns) + list(colones.keys()), fill_value=0)

display(df.head(100))

In [0]:
#Operations
max_maturity = df["maturity"].max()
df = df.assign(cash_flow_1=lambda x: x.cash_flow_1 + x["versement"])

for i in range(2,max_maturity):
    df = df.assign(**{f"cash_flow_{i}": lambda x, i=i: x[f"cash_flow_{i-1}"] - x["versement"] + x[f"cash_flow_{i-1}"] * x["tx_interet"]})

display(df.head(5))


In [0]:
#25-06-30
def projection_cf_mortgage(data):
 
    # Initialize the DataFrame
    cfs = np.zeros((len(data), 1200))
   
    # Iterate over each row in the data
    for j in range(len(data)):
        if data.iloc[j]['maturity']>0:
            tx_interet = data.iloc[j]['taux_hyp'] / data.iloc[j]['frequence']
            maturity = data.iloc[j]['maturity']
            pay_cap = data.iloc[j]['versement']
            notionnel_cul = np.zeros(maturity + 1)
            notionnel_cul[0] = data.iloc[j]['solde_capital']
           
            for k in range(maturity):
                notionnel_cul[k + 1] = notionnel_cul[k] - pay_cap + notionnel_cul[k]*tx_interet
                cfs[j, k] += pay_cap
 
            cfs[j, maturity - 1] += notionnel_cul[maturity]
 
    # Convert the NumPy array to a DataFrame
    cfs_df = pd.DataFrame(cfs, columns=[f'Cash Flow {i}' for i in range(1, cfs.shape[1] + 1)])
   
    # Concatenate the data with the cash flows
    result = pd.concat([data.reset_index(drop=True), cfs_df], axis=1)
    return result